In [203]:
# THIS FILE AGGREGATES BPT DATA AND MERGES TOGETHER TO ENABLE FURTHER PROCESSING
# NB it accepts both xlsx and xlsb files as input. But binary files seem to run quicker.

import pandas as pd
import numpy as np
import glob
import os

In [237]:
# These are the O drive location where we'll save our files
path3 = r"C:\Users\Maria.diapouli\OneDrive - OFWAT\Python\validation_tool\Excel Aggregation\Data"


In [238]:
# change the current directory
# to specified directory
os.chdir(path3)

In [ ]:
%%time 
# Create aggregated f_outputs sheets from companies' files

# Create an empty dataframes
df = pd.DataFrame()

#open all files in a directory
#for file in glob.glob(r"*.xls*"):  # glob is a way of getting all the files of a certain type
#open a specific file 
for file in glob.glob('SBB_PR24-BP-tables-V6-Publish_Amended.xlsx'):  
    acronym = file[0:3]  # Get the three letter prefix we have added. This assumes that each file has the acronym of the company added at the beginning
    xl = pd.ExcelFile(file)  # Define the excel file
    worksheets = xl.sheet_names  # Get the list of worksheets in the file
    for i in worksheets:  # For each worksheet...
        if "fOut" in i:  # ...if the workseet contains the text "fOut" then...
            df_temp = pd.read_excel(file, sheet_name= i, skiprows=1)  # Put the data into a temporary dataframe
            df_temp = df_temp.iloc[1:]  # Chop off the top row (in our file it is unused)
            # headers_temp = set(df_temp.columns)  # In development this and the next row were used to ensure the df would ultimately have all the right headers
            # headers = headers.union(headers_temp)
            df_temp['Acronym'] = acronym  # Add in the acronym name so we we can identify the company the data relates to
            df_temp['Sheet_name'] = i  # Add in the name of the worksheet, so we locate any issues more easily
            print(acronym + " " + i + " completed ok")
            df = pd.concat([df, df_temp])  # Add the temporary df to the master dataframe


In [235]:
df = df[["Acronym", "Reference", "Sheet_name", "Item description", "Unit", "Model", 'Constant', # Reorder the years columns
         "2011-12", "2012-13", "2013-14", "2014-15", "2015-16", "2016-17", "2017-18", "2018-19", "2019-20", "2020-21", "2021-22", "2022-23", "2023-24", "2024-25", "2025-26", "2026-27", "2027-28", "2028-29", "2029-30", "2030-31", "2031-32", "2032-33", "2033-34", "2034-35",
         "2020-25", "2025-30", "2030-35", "2035-40", "2040-45", "2045-50",
         '2039-40', '2044-45', '2025-55', '2049-50']]

df.reset_index(inplace=True, drop=True)

In [222]:
%%time 
# Export the results to excel, pickle and sqlite3
#df.to_excel(r".\Aggregated outputs\WSX.xlsx", sheet_name="F_Outputs")

CPU times: total: 0 ns
Wall time: 0 ns


In [236]:
%%time 
#prints the wall time for the entire cell for Ipython and jupyter
#Find Duplicate rows, do not keep first instance of duplicate into original file 
duplicateByRow = df[df.duplicated(subset=['Acronym', 'Reference'],keep=False)]

# Export the results to excel
duplicateByRow.to_excel(r".\Aggregated outputs\duplicated_SBB.xlsx", sheet_name="Duplicates")



CPU times: total: 2.42 s
Wall time: 2.58 s


In [226]:
%%time 
# Dropping ALL duplicate values, modify the exisitng DataFrame
df.drop_duplicates(subset=['Reference'],keep=False,inplace=True)
df.to_excel(r".\Aggregated outputs\Files_No_Duplicates_Batch_3\no_duplicates_WSX.xlsx", sheet_name="F_Outputs")

CPU times: total: 7.3 s
Wall time: 7.66 s
